In [4]:
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.snowball import SnowballStemmer
import nltk
import numpy as np
import spacy
import pandas as pd
from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer

from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
from scipy.special import softmax
import numpy as np
from array import array

nlp = spacy.load('en_core_web_lg')

df = pd.read_csv('./scraper/archive/javascript.csv', names=["POSTID", "POSTLINK", "POSTTITLE", "POSTBODY", "POSTDATE", "POSTVOTECOUNTS", "COMMENTID", "COMMENTSCORE", "COMMENTUSERNAME", "COMMENTTEXT", "COMMENTDATETIME", "ANSWERID", "ANSWERTEXT", "ANSWERBODY", "ANSWERDATETIME", "ANSWERVOTECOUNTS", "ANSWERCMTID", "ANSWERCMTTEXT", "ANSWERCMTBODY", "ANSWERCMTDATETIME", "ANSWERCMTVOTECOUNTS" , "TYPE"])

print(df.shape)
display(df.head())

print("\n")
print("All Columns DF")
df.columns

(101724, 22)


,POSTID,POSTLINK,POSTTITLE,POSTBODY,POSTDATE,POSTVOTECOUNTS,COMMENTID,COMMENTSCORE,COMMENTUSERNAME,COMMENTTEXT,...,ANSWERTEXT,ANSWERBODY,ANSWERDATETIME,ANSWERVOTECOUNTS,ANSWERCMTID,ANSWERCMTTEXT,ANSWERCMTBODY,ANSWERCMTDATETIME,ANSWERCMTVOTECOUNTS,TYPE
0,76144358,https://stackoverflow.com/questions/76144358/i...,I'm creating a color blender using Adobe Anima...,\n \n<p>I'm using Adobe Animate...,"('2023-05-01 01:47:04Z',)","('-1',)",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,post
1,36280818,https://stackoverflow.com/questions/36280818/h...,How to convert file to base64 in JavaScript?,\n \n<p><strong>UPD</strong> Ty...,"('2016-03-29 09:49:26Z',)","('355',)",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,post
2,36280818,https://stackoverflow.com/questions/36280818/h...,How to convert file to base64 in JavaScript?,\n \n<p><strong>UPD</strong> Ty...,"('2016-03-29 09:49:26Z',)","('355',)",NaN,NaN,NaN,NaN,...,Try the solution using the FileReader class:\n...,('\n<p>Try the solution using the <code>FileRe...,"2017-06-09 19:16:52Z, License: CC BY-SA 3.0","('429',)",NaN,NaN,NaN,NaN,NaN,answer
3,36280818,https://stackoverflow.com/questions/36280818/h...,How to convert file to base64 in JavaScript?,\n \n<p><strong>UPD</strong> Ty...,"('2016-03-29 09:49:26Z',)","('355',)",NaN,NaN,NaN,NaN,...,Try the solution using the FileReader class:\n...,('\n<p>Try the solution using the <code>FileRe...,"2017-06-09 19:16:52Z, License: CC BY-SA 3.0","('429',)","('75927236',)",read more about FileReader API: developer.mozi...,('\n \n <span cl...,"Jun 9, 2017 at 19:16","('3',)",answer_comment
4,36280818,https://stackoverflow.com/questions/36280818/h...,How to convert file to base64 in JavaScript?,\n \n<p><strong>UPD</strong> Ty...,"('2016-03-29 09:49:26Z',)","('355',)",NaN,NaN,NaN,NaN,...,Try the solution using the FileReader class:\n...,('\n<p>Try the solution using the <code>FileRe...,"2017-06-09 19:16:52Z, License: CC BY-SA 3.0","('429',)","('81338095',)",I tried to use return reader.result from the g...,('\n \n <span cl...,"Nov 9, 2017 at 5:35","('20',)",answer_comment




All Columns DF


Index(['POSTID', 'POSTLINK', 'POSTTITLE', 'POSTBODY', 'POSTDATE',
       'POSTVOTECOUNTS', 'COMMENTID', 'COMMENTSCORE', 'COMMENTUSERNAME',
       'COMMENTTEXT', 'COMMENTDATETIME', 'ANSWERID', 'ANSWERTEXT',
       'ANSWERBODY', 'ANSWERDATETIME', 'ANSWERVOTECOUNTS', 'ANSWERCMTID',
       'ANSWERCMTTEXT', 'ANSWERCMTBODY', 'ANSWERCMTDATETIME',
       'ANSWERCMTVOTECOUNTS', 'TYPE'],
      dtype='object')

In [5]:
unique_values = df['POSTTITLE'].unique()
print("There are a total of {} unique titles".format(len(unique_values)))

query = 'What is the correct way to check for string equality in JavaScript? ' 
''' case sensitive '''
''' convert date to string '''
''' Event loop '''
''' What is the correct way to check for string equality in JavaScript? '''
''' How to compare two strings value '''

There are a total of 16262 unique titles


' How to compare two strings value '

## String Similarity

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("roberta-large-mnli")
model = AutoModelForSequenceClassification.from_pretrained("roberta-large-mnli")

def test_entailment(text1, text2):
    batch = tokenizer(text1, text2, return_tensors='pt').to(model.device)
    with torch.no_grad():
        proba = torch.softmax(model(**batch).logits, -1)
    return proba.cpu().numpy()[0, model.config.label2id['ENTAILMENT']]

def test_equivalence(text1, text2):
    return test_entailment(text1, text2) * test_entailment(text2, text1) * 100

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
def calPartialRatio (str1, str2):
    return fuzz.partial_ratio(str1, str2)

def calTokenSortRatio (str1, str2):
    return fuzz.token_sort_ratio(str1, str2)
  
def calTokenSetRatio (str1, str2):
    return fuzz.token_set_ratio(str1, str2)

fuzzywuzzy_df = pd.DataFrame(columns=['postId', 'title', 'partial_ratio', 'token_sort_ratio', 'token_set_ratio', 'bert_similarity'])
for unique_value in unique_values:
    # find the postId of the unique value
    postId = df[df['POSTTITLE'] == unique_value]['POSTID'].tolist()[0]
    fuzzywuzzy_df = pd.concat([fuzzywuzzy_df, pd.DataFrame({'postId': postId , 'title': unique_value, 'partial_ratio': calPartialRatio(unique_value, query), 'token_sort_ratio': calTokenSortRatio(unique_value, query), 'token_set_ratio': calTokenSetRatio(unique_value, query)}, index=[0])] )

matchesPR = fuzzywuzzy_df.sort_values(by=['partial_ratio'], ascending=False)
matchesPR = matchesPR[matchesPR['partial_ratio'] > 50]

matchesTS = fuzzywuzzy_df.sort_values(by=['token_sort_ratio'], ascending=False)
matchesTS = matchesTS[matchesTS['token_sort_ratio'] > 50]

matchesTSE = fuzzywuzzy_df.sort_values(by=['token_set_ratio'], ascending=False)
matchesTSE = matchesTSE[matchesTSE['token_set_ratio'] > 80]

Top10MathesMerged = pd.concat([matchesTS, matchesPR]).drop_duplicates()
Top10MathesMerged['average'] = (Top10MathesMerged['token_sort_ratio'] + Top10MathesMerged['partial_ratio']) / 2
Top10MathesMerged = Top10MathesMerged.sort_values(by=['average'], ascending=False)

Top10MathesMerged = Top10MathesMerged[Top10MathesMerged['average'] > 50]
display(matchesTSE)
print("{} number of titles found".format(len(matchesTSE)))

,postId,title,partial_ratio,token_sort_ratio,token_set_ratio,bert_similarity
0,5914020,javascript date to string,52,51,89,NaN
0,5717093,Check if a JavaScript string is a URL,47,58,85,NaN
0,75571616,Quiz in Javascript,78,36,84,NaN
0,26100218,Reverse String in JavaScript,68,43,83,NaN
0,8900652,What does % do in JavaScript?,59,48,82,NaN
0,17880476,Joins in Javascript,74,38,81,NaN
0,1646698,What is the 'new' keyword in JavaScript?,50,54,81,NaN
0,3001866,Call the id in JavaScript,68,48,81,NaN
0,4059147,Check if a variable is a string in JavaScript,65,63,81,NaN
0,6815903,What is the correct way of code comments in Ja...,59,70,81,NaN


12 number of titles found


In [8]:
df = df[df['POSTID'].isin(matchesTSE['postId'])]

# populate df with token set ratio value
merged_df = pd.merge(df, matchesTSE, left_on='POSTID', right_on='postId', how='left')

merged_df.to_csv('convertdatetostr-tsematches.csv')

# Input of tsematches

In [9]:
df = pd.read_csv('convertdatetostr-tsematches.csv') 

In [10]:

df['POSTDATE'] = df['POSTDATE'].str.replace(r"(", '')
df['POSTDATE'] = df['POSTDATE'].str.replace(r")", '')
df['POSTDATE'] = df['POSTDATE'].str.replace(r"'", '')
df['POSTDATE'] = df['POSTDATE'].str.replace(r",", '')

df['POSTVOTECOUNTS'] = df['POSTVOTECOUNTS'].str.replace(r"(", '')
df['POSTVOTECOUNTS'] = df['POSTVOTECOUNTS'].str.replace(r")", '')
df['POSTVOTECOUNTS'] = df['POSTVOTECOUNTS'].str.replace(r"'", '')
df['POSTVOTECOUNTS'] = df['POSTVOTECOUNTS'].str.replace(r",", '')
df["POSTVOTECOUNTS"] = df["POSTVOTECOUNTS"].astype(int)

In [11]:
from datetime import datetime
date_format = "%Y-%m-%d %H:%M:%SZ"

df["POSTDATE"] = df["POSTDATE"].apply(lambda x: datetime.strptime(x, date_format))


In [12]:
def calculate_time_score(df, date_column):
    current_time = datetime.now()
    df[date_column] = pd.to_datetime(df[date_column])  # Convert the date column to datetime
    
    # Calculate the range of time differences
    time_diffs = (current_time - df[date_column]).dt.total_seconds() / 60
    min_time_diff = np.min(time_diffs)
    max_time_diff = np.max(time_diffs)
    
    # Calculate the time score for each row
    time_scores = 100 - ((time_diffs - min_time_diff) / (max_time_diff - min_time_diff)) * 100
    time_scores = np.maximum(np.minimum(time_scores, 100), 1)  # Ensure scores are within the range of 1 to 100
    
    return time_scores

time_scores = calculate_time_score(df, "POSTDATE")

df["time_score"] = time_scores

In [13]:
def calculate_vote_count_score(df, vote_column):
    min_vote_count = df[vote_column].min()
    max_vote_count = df[vote_column].max()

    print("{} min vote count, {} max vote count".format(min_vote_count, max_vote_count))

    vote_counts = df[vote_column]
    vote_count_scores = 100 - ((vote_counts - min_vote_count) / (max_vote_count - min_vote_count)) * 100
    vote_count_scores = np.maximum(np.minimum(vote_count_scores, 100), 1)  # Ensure scores are within the range of 1 to 100

    return vote_count_scores
  
  
votecount_scores = calculate_vote_count_score(df, "POSTVOTECOUNTS")
df['vote_score'] = votecount_scores

0 min vote count, 2639 max vote count


In [14]:
sorted = df.sort_values(by=['vote_score'], ascending=False)
sorted

,Unnamed: 0,POSTID,POSTLINK,POSTTITLE,POSTBODY,POSTDATE,POSTVOTECOUNTS,COMMENTID,COMMENTSCORE,COMMENTUSERNAME,...,ANSWERCMTVOTECOUNTS,TYPE,postId,title,partial_ratio,token_sort_ratio,token_set_ratio,bert_similarity,time_score,vote_score
171,171,75571616,https://stackoverflow.com/questions/75571616/q...,Quiz in Javascript,\n \n<p>I'm quite new to progra...,2023-02-26 11:18:47,0,NaN,NaN,NaN,...,NaN,post,75571616,Quiz in Javascript,78,36,84,NaN,100.000000,100.0
242,242,74497558,https://stackoverflow.com/questions/74497558/a...,Alert in Javascript,"\n \n<p>On my order site, I hav...",2022-11-19 04:29:33,0,comment-131507612,NaN,Sebastian Simon,...,NaN,post_comment,74497558,Alert in Javascript,79,40,81,NaN,97.960294,100.0
233,233,3001866,https://stackoverflow.com/questions/3001866/ca...,Call the id in JavaScript,\n \n<p>I have a span like this...,2010-06-08 22:43:21,0,NaN,NaN,NaN,...,NaN,post,3001866,Call the id in JavaScript,68,48,81,NaN,4.561801,100.0
234,234,3001866,https://stackoverflow.com/questions/3001866/ca...,Call the id in JavaScript,\n \n<p>I have a span like this...,2010-06-08 22:43:21,0,comment-3066613,NaN,Asim Zaidi,...,NaN,post_comment,3001866,Call the id in JavaScript,68,48,81,NaN,4.561801,100.0
235,235,3001866,https://stackoverflow.com/questions/3001866/ca...,Call the id in JavaScript,\n \n<p>I have a span like this...,2010-06-08 22:43:21,0,comment-3066636,NaN,Marcel Korpel,...,NaN,post_comment,3001866,Call the id in JavaScript,68,48,81,NaN,4.561801,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,139,4059147,https://stackoverflow.com/questions/4059147/ch...,Check if a variable is a string in JavaScript,\n \n<p>How can I determine whe...,2010-10-30 14:36:34,2639,NaN,NaN,NaN,...,"('4',)",answer_comment,4059147,Check if a variable is a string in JavaScript,65,63,81,NaN,7.513209,1.0
140,140,4059147,https://stackoverflow.com/questions/4059147/ch...,Check if a variable is a string in JavaScript,\n \n<p>How can I determine whe...,2010-10-30 14:36:34,2639,NaN,NaN,NaN,...,"('4',)",answer_comment,4059147,Check if a variable is a string in JavaScript,65,63,81,NaN,7.513209,1.0
141,141,4059147,https://stackoverflow.com/questions/4059147/ch...,Check if a variable is a string in JavaScript,\n \n<p>How can I determine whe...,2010-10-30 14:36:34,2639,NaN,NaN,NaN,...,"('1',)",answer_comment,4059147,Check if a variable is a string in JavaScript,65,63,81,NaN,7.513209,1.0
142,142,4059147,https://stackoverflow.com/questions/4059147/ch...,Check if a variable is a string in JavaScript,\n \n<p>How can I determine whe...,2010-10-30 14:36:34,2639,NaN,NaN,NaN,...,"('1',)",answer_comment,4059147,Check if a variable is a string in JavaScript,65,63,81,NaN,7.513209,1.0


In [15]:
fuzzy_weightage = 0.8
time_weightage = 0.2

In [16]:
results = []
for index, row in df.iterrows():
    similarity_score = row['token_set_ratio']
    time_score = row['time_score']
    
    weighted_score = (fuzzy_weightage * similarity_score) + (time_weightage * time_score)
    
    results.append({"POSTID": row["POSTID"], "weighted_score": weighted_score})

results.sort(key=lambda x: x["weighted_score"], reverse=True)

results_df = pd.DataFrame(results)

# Print the sorted results DataFrame
print(results_df)

       POSTID  weighted_score
0    75571616       87.200000
1    74497558       84.392059
2    74497558       84.392059
3    74497558       84.392059
4    26100218       73.777932
..        ...             ...
245   1646698       65.000000
246   1646698       65.000000
247   1646698       65.000000
248   1646698       65.000000
249   1646698       65.000000

[250 rows x 2 columns]


In [17]:
df[df['POSTID'] == results_df['POSTID'][5]]

,Unnamed: 0,POSTID,POSTLINK,POSTTITLE,POSTBODY,POSTDATE,POSTVOTECOUNTS,COMMENTID,COMMENTSCORE,COMMENTUSERNAME,...,ANSWERCMTVOTECOUNTS,TYPE,postId,title,partial_ratio,token_sort_ratio,token_set_ratio,bert_similarity,time_score,vote_score
0,0,26100218,https://stackoverflow.com/questions/26100218/r...,Reverse String in JavaScript,\n \n<p>I wrote a JS constructo...,2014-09-29 12:41:59,2,NaN,NaN,NaN,...,NaN,post,26100218,Reverse String in JavaScript,68,43,83,NaN,36.889661,99.924214
1,1,26100218,https://stackoverflow.com/questions/26100218/r...,Reverse String in JavaScript,\n \n<p>I wrote a JS constructo...,2014-09-29 12:41:59,2,comment-40900823,4.0,Frédéric Hamidi,...,NaN,post_comment,26100218,Reverse String in JavaScript,68,43,83,NaN,36.889661,99.924214
2,2,26100218,https://stackoverflow.com/questions/26100218/r...,Reverse String in JavaScript,\n \n<p>I wrote a JS constructo...,2014-09-29 12:41:59,2,comment-40900875,4.0,Frédéric Hamidi,...,NaN,post_comment,26100218,Reverse String in JavaScript,68,43,83,NaN,36.889661,99.924214
3,3,26100218,https://stackoverflow.com/questions/26100218/r...,Reverse String in JavaScript,\n \n<p>I wrote a JS constructo...,2014-09-29 12:41:59,2,comment-40900914,NaN,user663031,...,NaN,post_comment,26100218,Reverse String in JavaScript,68,43,83,NaN,36.889661,99.924214
4,4,26100218,https://stackoverflow.com/questions/26100218/r...,Reverse String in JavaScript,\n \n<p>I wrote a JS constructo...,2014-09-29 12:41:59,2,comment-40900984,NaN,user663031,...,NaN,post_comment,26100218,Reverse String in JavaScript,68,43,83,NaN,36.889661,99.924214
5,5,26100218,https://stackoverflow.com/questions/26100218/r...,Reverse String in JavaScript,\n \n<p>I wrote a JS constructo...,2014-09-29 12:41:59,2,comment-40901194,NaN,user663031,...,NaN,post_comment,26100218,Reverse String in JavaScript,68,43,83,NaN,36.889661,99.924214


In [18]:
df[df['POSTID'] == results_df['POSTID'][len(results_df) - 1]]

,Unnamed: 0,POSTID,POSTLINK,POSTTITLE,POSTBODY,POSTDATE,POSTVOTECOUNTS,COMMENTID,COMMENTSCORE,COMMENTUSERNAME,...,ANSWERCMTVOTECOUNTS,TYPE,postId,title,partial_ratio,token_sort_ratio,token_set_ratio,bert_similarity,time_score,vote_score
172,172,1646698,https://stackoverflow.com/questions/1646698/wh...,What is the 'new' keyword in JavaScript?,\n \n<p>The <code>new</code> ke...,2009-10-29 21:32:45,1912,NaN,NaN,NaN,...,NaN,post,1646698,What is the 'new' keyword in JavaScript?,50,54,81,NaN,1.0,27.548314
173,173,1646698,https://stackoverflow.com/questions/1646698/wh...,What is the 'new' keyword in JavaScript?,\n \n<p>The <code>new</code> ke...,2009-10-29 21:32:45,1912,comment-1517440,13.0,Chetan S,...,NaN,post_comment,1646698,What is the 'new' keyword in JavaScript?,50,54,81,NaN,1.0,27.548314
174,174,1646698,https://stackoverflow.com/questions/1646698/wh...,What is the 'new' keyword in JavaScript?,\n \n<p>The <code>new</code> ke...,2009-10-29 21:32:45,1912,comment-94796557,1.0,MartianMartian,...,NaN,post_comment,1646698,What is the 'new' keyword in JavaScript?,50,54,81,NaN,1.0,27.548314
175,175,1646698,https://stackoverflow.com/questions/1646698/wh...,What is the 'new' keyword in JavaScript?,\n \n<p>The <code>new</code> ke...,2009-10-29 21:32:45,1912,NaN,NaN,NaN,...,NaN,answer,1646698,What is the 'new' keyword in JavaScript?,50,54,81,NaN,1.0,27.548314
176,176,1646698,https://stackoverflow.com/questions/1646698/wh...,What is the 'new' keyword in JavaScript?,\n \n<p>The <code>new</code> ke...,2009-10-29 21:32:45,1912,NaN,NaN,NaN,...,"('53',)",answer_comment,1646698,What is the 'new' keyword in JavaScript?,50,54,81,NaN,1.0,27.548314
177,177,1646698,https://stackoverflow.com/questions/1646698/wh...,What is the 'new' keyword in JavaScript?,\n \n<p>The <code>new</code> ke...,2009-10-29 21:32:45,1912,NaN,NaN,NaN,...,"('12',)",answer_comment,1646698,What is the 'new' keyword in JavaScript?,50,54,81,NaN,1.0,27.548314
178,178,1646698,https://stackoverflow.com/questions/1646698/wh...,What is the 'new' keyword in JavaScript?,\n \n<p>The <code>new</code> ke...,2009-10-29 21:32:45,1912,NaN,NaN,NaN,...,"('14',)",answer_comment,1646698,What is the 'new' keyword in JavaScript?,50,54,81,NaN,1.0,27.548314
179,179,1646698,https://stackoverflow.com/questions/1646698/wh...,What is the 'new' keyword in JavaScript?,\n \n<p>The <code>new</code> ke...,2009-10-29 21:32:45,1912,NaN,NaN,NaN,...,"('11',)",answer_comment,1646698,What is the 'new' keyword in JavaScript?,50,54,81,NaN,1.0,27.548314
180,180,1646698,https://stackoverflow.com/questions/1646698/wh...,What is the 'new' keyword in JavaScript?,\n \n<p>The <code>new</code> ke...,2009-10-29 21:32:45,1912,NaN,NaN,NaN,...,"('7',)",answer_comment,1646698,What is the 'new' keyword in JavaScript?,50,54,81,NaN,1.0,27.548314
181,181,1646698,https://stackoverflow.com/questions/1646698/wh...,What is the 'new' keyword in JavaScript?,\n \n<p>The <code>new</code> ke...,2009-10-29 21:32:45,1912,NaN,NaN,NaN,...,NaN,answer,1646698,What is the 'new' keyword in JavaScript?,50,54,81,NaN,1.0,27.548314
